In [33]:
import json
import numpy as np

# Function to read JSON file and parse results
def read_results(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        results = json.load(f)
    return results

# Function to compute mean metrics over the classes for each run
def compute_mean_metrics(results):
    analysis = {}
    for config, data in results.items():
        few_shot_data = data.get("few_shot", {})
        analysis[config] = {}
        for shot, shot_data in few_shot_data.items():
            analysis[config][shot] = {"train": {}, "test": {}}
            for res_type in ["train", "test"]:
                res_data = shot_data.get(res_type, {})
                class_metrics = {k: v for k, v in res_data.items() if k != "overall"}
                
                # Get all unique run numbers
                run_numbers = set(run for metrics in class_metrics.values() for run in metrics.keys())

                for run in run_numbers:
                    # Initialize dictionary to hold the sums and counts for each metric
                    metric_sums = {}
                    metric_counts = {}

                    # Collect metric values for each class
                    for class_name, metrics in class_metrics.items():
                        if run in metrics:
                            run_metrics = metrics[run]
                            for metric, value in run_metrics.items():
                                if not np.isnan(value) and value != -1:
                                    if metric not in metric_sums:
                                        metric_sums[metric] = 0
                                        metric_counts[metric] = 0
                                    metric_sums[metric] += value
                                    metric_counts[metric] += 1

                    # Compute mean for each metric for this run
                    mean_metrics = {metric: (metric_sums[metric] / metric_counts[metric] if metric_counts[metric] > 0 else float('nan')) for metric in metric_sums}

                    # Multiply the metrics by 100
                    mean_metrics = {metric: value * 100 for metric, value in mean_metrics.items()}
                    
                    if run not in analysis[config][shot][res_type]:
                        analysis[config][shot][res_type][run] = {}
                    analysis[config][shot][res_type][run] = {
                        "mean_metrics": mean_metrics
                    }

    return analysis

# Path to the JSON results file
file_path = 'evaluation_results.json'

# Read the results from the JSON file
results = read_results(file_path)

# Compute the analysis
mean_results = compute_mean_metrics(results)

# Print the analysis
import pprint
pprint.pprint(mean_results)


{'fcos_PVTV2B2LI_FPNRETINANET_XQSABGA_DIOR.yaml': {'10_shot': {'test': {'0': {'mean_metrics': {'AP': 2.0818186380394046,
                                                                                               'AP50': 4.940412979466116,
                                                                                               'AP75': 1.5630441351571482,
                                                                                               'APl': 4.145027740529287,
                                                                                               'APm': 2.6528321170451497,
                                                                                               'APs': 0.6717463642751833}},
                                                                        '1': {'mean_metrics': {'AP': 2.323729308134053,
                                                                                               'AP50': 5.663571241633669,
                         

In [36]:
import numpy as np

# New evaluation results to be added
new_evaluation_results = {
    "faster_rcnn_FCT_DOTA.yaml": {
        "10_shot": {
            "train": {
                "0": {
                    "mean_metrics": {
                        "AP": 28.3912,
                        "AP50": 49.9011,
                        "AP75": 28.2457,
                        "APs": 22.9618,
                        "APm": 54.6035,
                        "APl": 55.3671
                    }
                }
            },
            "test": {
                "0": {
                    "mean_metrics": {
                        "AP": 17.5446,
                        "AP50": 32.6972,
                        "AP75": 16.6045,
                        "APs": 16.2349,
                        "APm": 19.7232,
                        "APl": 46.2229
                    }
                }
            }
        }
    },
    "faster_rcnn_FCT_DIOR.yaml": {
        "10_shot": {
            "train": {
                "0": {
                    "mean_metrics": {
                        "AP": 51.8869,
                        "AP50": 69.5572,
                        "AP75": 57.7049,
                        "APs": 10.8314,
                        "APm": 51.6611,
                        "APl": 88.5412
                    }
                }
            },
            "test": {
                "0": {
                    "mean_metrics": {
                        "AP": 22.0111,
                        "AP50": 38.5204,
                        "AP75": 23.5506,
                        "APs": 0.8578,
                        "APm": 21.7535,
                        "APl": 66.0012
                    }
                }
            }
        }
    }
}

# Function to add new evaluation results to existing analysis
def add_new_results_to_analysis(analysis, new_results):
    for config, config_data in new_results.items():
        if config not in analysis:
            analysis[config] = {}
        for shot, shot_data in config_data.items():
            if shot not in analysis[config]:
                analysis[config][shot] = {"train": {}, "test": {}}
            for res_type, runs in shot_data.items():
                if res_type not in analysis[config][shot]:
                    analysis[config][shot][res_type] = {"mean_metrics": {}}
                for run_id, run_data in runs.items():
                    analysis[config][shot][res_type][run_id] = run_data

# Add the new evaluation results to the analysis
add_new_results_to_analysis(mean_results, new_evaluation_results)
pprint.pprint(mean_results)

{'faster_rcnn_FCT_DIOR.yaml': {'10_shot': {'test': {'0': {'mean_metrics': {'AP': 22.0111,
                                                                           'AP50': 38.5204,
                                                                           'AP75': 23.5506,
                                                                           'APl': 66.0012,
                                                                           'APm': 21.7535,
                                                                           'APs': 0.8578}}},
                                           'train': {'0': {'mean_metrics': {'AP': 51.8869,
                                                                            'AP50': 69.5572,
                                                                            'AP75': 57.7049,
                                                                            'APl': 88.5412,
                                                                            'APm':

In [37]:
import numpy as np

# Function to compute additional statistics over the runs
def compute_run_statistics(analysis):
    stats = {}
    for config, data in analysis.items():
        stats[config] = {}
        for shot, shot_data in data.items():
            stats[config][shot] = {"train": {}, "test": {}}
            for res_type in ["train", "test"]:
                metrics = shot_data[res_type]
                
                # Collect all metrics values
                metrics_collection = {metric: [] for metric in metrics["0"]["mean_metrics"]}
                for run_id, run_data in metrics.items():
                    for metric, value in run_data["mean_metrics"].items():
                        metrics_collection[metric].append(value)
                
                # Calculate statistics
                stats[config][shot][res_type] = {
                    metric: {
                        "mean": np.mean(values),
                        "min": np.min(values),
                        "max": np.max(values),
                        "variance": np.var(values)
                    }
                    for metric, values in metrics_collection.items()
                }
    return stats

# Compute the extended analysis
run_statistics = compute_run_statistics(mean_results)

# Print the extended analysis
import pprint
pprint.pprint(run_statistics)


{'faster_rcnn_FCT_DIOR.yaml': {'10_shot': {'test': {'AP': {'max': 22.0111,
                                                           'mean': 22.0111,
                                                           'min': 22.0111,
                                                           'variance': 0.0},
                                                    'AP50': {'max': 38.5204,
                                                             'mean': 38.5204,
                                                             'min': 38.5204,
                                                             'variance': 0.0},
                                                    'AP75': {'max': 23.5506,
                                                             'mean': 23.5506,
                                                             'min': 23.5506,
                                                             'variance': 0.0},
                                                    'APl': {'max': 66.0012,

In [50]:
import pandas as pd

# Function to extract dataset name from config file name and clean the config name
def extract_and_clean_config(config_file):
    parts = config_file.split('_')
    dataset_name = parts[-1].replace('.yaml', '')
    cleaned_config = '_'.join(parts[:-1])
    return cleaned_config, dataset_name

# Function to convert analysis to a pandas DataFrame
def analysis_to_dataframe(analysis):
    records = []
    for config, config_data in analysis.items():
        cleaned_config, dataset_name = extract_and_clean_config(config)
        for shot, shot_data in config_data.items():
            for res_type, res_data in shot_data.items():
                for metric, stats in res_data.items():
                    record = {
                        "Config": cleaned_config,
                        "Dataset": dataset_name,
                        "Shot": shot,
                        "Type": res_type,
                        "Metric": metric,
                        "Mean": stats['mean'],
                        "Variance": stats['variance'],
                        "Min": stats['min'],
                        "Max": stats['max']
                    }
                    records.append(record)
    
    df = pd.DataFrame(records)
    df.set_index(['Dataset', 'Shot', 'Config', 'Type', 'Metric'], inplace=True)
    df = df.unstack(level=-1)  # Move the metric level to columns
    return df

# Convert the analysis to a DataFrame
df = analysis_to_dataframe(run_statistics)

# Print or save the DataFrame
print("Multi-Index DataFrame:")
(df).round(2)


Multi-Index DataFrame:


Mean         \
Metric                                                        AP   AP50   
Dataset Shot    Config                              Type                  
DIOR    10_shot faster_rcnn_FCT                     test   22.01  38.52   
                                                    train  51.89  69.56   
                fcos_PVTV2B2LI_FPNRETINANET_XQSABGA test    2.20   5.30   
                                                    train   1.13   1.77   
                fcos_R50_FPNRETINANET_XQSABGA       test    3.40   7.12   
                                                    train   0.75   1.23   
DOTA    10_shot faster_rcnn_FCT                     test   17.54  32.70   
                                                    train  28.39  49.90   
                fcos_PVTV2B2LI_FPNRETINANET_XQSABGA test    1.10   2.08   
                                                    train   0.81   1.91   

                                                                         \
Metric                                                      AP75    APl   
Dataset Shot    Config                              Type                  
DIOR    10_shot faster_rcnn_FCT                     test   23.55  66.00   
                                                    train  57.70  88.54   
                fcos_PVTV2B2LI_FPNRETINANET_XQSABGA test    1.56   4.29   
                                                    train   1.25   3.12   
                fcos_R50_FPNRETINANET_XQSABGA       test    2.96   6.57   
                                                    train   0.75   2.29   
DOTA    10_shot faster_rcnn_FCT                     test   16.60  46.22   
                                                    train  28.25  55.37   
                fcos_PVTV2B2LI_FPNRETINANET_XQSABGA test    1.04   2.56   
                                                    train   0.63   1.88   

                                                                         \
Metric                                                       APm    APs   
Dataset Shot    Config                              Type                  
DIOR    10_shot faster_rcnn_FCT                     test   21.75   0.86   
                                                    train  51.66  10.83   
                fcos_PVTV2B2LI_FPNRETINANET_XQSABGA test    2.86   0.67   
                                                    train   0.56   0.18   
                fcos_R50_FPNRETINANET_XQSABGA       test    3.46   0.31   
                                                    train   0.42   0.12   
DOTA    10_shot faster_rcnn_FCT                     test   19.72  16.23   
                                                    train  54.60  22.96   
                fcos_PVTV2B2LI_FPNRETINANET_XQSABGA test    2.15   0.70   
                                                    train   1.21   0.25   

                                                          Variance        \
Metric                                                          AP  AP50   
Dataset Shot    Config                              Type                   
DIOR    10_shot faster_rcnn_FCT                     test      0.00  0.00   
                                                    train     0.00  0.00   
                fcos_PVTV2B2LI_FPNRETINANET_XQSABGA test      0.01  0.13   
                                                    train     0.08  0.25   
                fcos_R50_FPNRETINANET_XQSABGA       test      0.00  0.00   
                                                    train     0.01  0.02   
DOTA    10_shot faster_rcnn_FCT                     test      0.00  0.00   
                                                    train     0.00  0.00   
                fcos_PVTV2B2LI_FPNRETINANET_XQSABGA test      0.01  0.01   
                                                    train     0.02  0.12   

                                                                       ...  \
Metric                                                     AP75  

In [56]:
# Function to highlight the maximum values in bold for each group
def highlight_max(df, group_levels, color='darkorange'):
    '''
    Highlight the maximum in a DataFrame group
    '''
    attr = f'font-weight: bold; font-style: italic; color: {color}'
    # Create a DataFrame with the same shape filled with empty strings
    styles = pd.DataFrame('', index=df.index, columns=df.columns)
    
    # Iterate over each group and apply the highlight
    for group_name, group_df in df.groupby(level=group_levels):
        # Find the max values for each metric within the group
        for metric in df.columns.levels[0]:
            for stat in df.columns.levels[1]:
                col = (metric, stat)
                if col in group_df.columns:
                    max_value = group_df[col].max()
                    is_max = group_df[col] == max_value
                    styles.loc[is_max.index, col] = np.where(is_max, attr, '')
    
    return styles

# Function to alternate the background colors for the "train" and "test" groups on index level
def alternate_background(df):
    styles = pd.DataFrame('', index=df.index, columns=df.columns)
    
    # Alternate background color for "train" and "test" groups
    colors = ['#333333', '#555555']
    
    group_counter = 0
    previous_group_type = None
    for (config, dataset, shot, group_type), group_df in df.groupby(level=['Config', 'Dataset', 'Shot', 'Type']):
        if group_type != previous_group_type:
            group_counter += 1
        bg_color = colors[group_counter % 2]
        for idx in group_df.index:
            styles.loc[idx, :] = f'background-color: {bg_color}'
        previous_group_type = group_type
    
    return styles

# Function to combine the styles
def combine_styles(highlighted, background):
    combined_styles = pd.DataFrame('', index=highlighted.index, columns=highlighted.columns)
    for col in highlighted.columns:
        for idx in highlighted.index:
            h_style = highlighted.loc[idx, col]
            b_style = background.loc[idx, col]
            combined_styles.loc[idx, col] = f'{h_style}; {b_style}' if h_style and b_style else h_style or b_style
    return combined_styles

# Apply both styles to the DataFrame
highlighted = highlight_max(df, group_levels=['Dataset', 'Shot', 'Type'])
# background = alternate_background(df)

# Combine the styles
combined_styles = combine_styles(highlighted, highlighted)

# Apply the combined styles to the DataFrame
styled_df = df.style.apply(lambda _: combined_styles, axis=None)

# Display the styled DataFrame
styled_df